### Text Selection Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
#path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Text Selection/Result/2-Text Selection Data Merge"

In [21]:
df_word = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx")
df_word = df_word.fillna("[]")
df_word = df_word.iloc[1000:2000,]
df_word

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,arka,69809,"['arka tarafta', 'arka tarafa', 'arka kapı', '...","['arka tarafta bir', 'bir arka kapı', 'arka ta...","['arka kapısı var mı', 'buranın arka kapısı va...","['buranın arka kapısı var mı', 'onlar sadece a...",['arka kapısında tv seti çalarken yakalamışlar...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...
1001,hoşuma,69776,"['hoşuma gitti', 'hepinizi kutsasın', 'bu hoşu...","['hiç hoşuma gitmedi', 'çok hoşuma gitti', 'bu...","['bu hiç hoşuma gitmedi', 'her şeyi bilen biri...","['bu iş hiç hoşuma gitmedi', 'bu benim de hoşu...","['bana şaplak atmana izin vermek hoşuma', 'şap...",['bana şaplak atmana izin vermek hoşuma gider'...,['tvde diğer ülkeleri falan görünce çok hoşuma...,['gece gelip çattı ve durum daha baştan hoşuma...,['büyük gece gelip çattı ve durum daha baştan ...
1002,üçüncü,69620,"['üçüncü bir', 'üçüncü kat', 've üçüncü', 'bu ...","['üçüncü ve dördüncü', 'bu hafta üçüncü', 'üçü...","['güney üçüncü sıra temiz', 'bu hafta üçüncü k...","['kuzey üçüncü sıra temiz sayım', 'üçüncü bir ...",['üçüncü mevkii yolcuları sağlık kontrolü için...,['üçüncü mevkii yolcuları sağlık kontrolü için...,['bütün üçüncü mevkii yolcuları sağlık kontrol...,['bütün üçüncü mevkii yolcuları sağlık kontrol...,['bütün üçüncü mevkii yolcuları sağlık kontrol...
1003,sırada,69586,"['sırada ne', 'o sırada', 'arada sırada', 'bu ...","['sırada ne var', 'sırada sen varsın', 'tam o ...","['peki sırada ne var', 'şimdi sırada ne var', ...","['top şu sırada sağ tarafta', 'arada sırada gü...","['bizi sırada ne tür bir macera', 'sırada ne t...","['sırada ne tür bir macera bekliyor acaba', 'b...",['bizi sırada ne tür bir macera bekliyor acaba...,['bu arkadaşın arkadaşa yapmayacağı şeyler lis...,['çünkü bu arkadaşın arkadaşa yapmayacağı şeyl...
1004,gidecek,69388,"['gidecek misin', 'yolunda gidecek', 'gidecek ...","['şey yolunda gidecek', 'çok hoşuna gidecek', ...","['geri getir olur mu', 'gidecek bir yerim yok'...","['gidecek başka bir yerim yok', 'gidecek bir y...","['burada bana eşlik etmen hoşuma gidecek', 'me...","['bu gece oraya bir adam uğradı mı', 'helikopt...",['adam gelecek ve sözde kayıp bavulunu alıp gi...,['gidecek birini rehin aldı ve ona kibar mı da...,['mahkemeye gidecek birini rehin aldı ve ona k...
...,...,...,...,...,...,...,...,...,...,...,...
1995,ihtiyacınız,33536,"['ihtiyacınız var', 'ihtiyacınız olursa', 'şey...","['ihtiyacınız var mı', 'bir şeye ihtiyacınız',...","['bir şeye ihtiyacınız olursa', 'yardıma ihtiy...","['bir şeye ihtiyacınız var mı', 'eğer bir şeye...","['başka bir şeye ihtiyacınız var mı', 'bir şey...",['bir şeye ihtiyacınız olursa bana haber verin...,['efendim bana ihtiyacınız yoksa bir süre kend...,['iş dün gecede kaldı başka bir şey olacağı yo...,['ne olursa olsun akıllı kişilere ihtiyacınız ...
1996,olmanı,33533,"['olmanı istiyorum', 'mutlu olmanı', 'olmanı i...","['mutlu olmanı istiyorum', 'göz kulak olmanı',...","['göz kulak olmanı istiyorum', 'senin mutlu ol...","['bana karşı dürüst olmanı istiyorum', 'sadece...","['senin hep böyle neşeli olmanı seviyorum', 'b...","['bu yüzük ile benim olmanı rica ediyorum', 'b...",['gereken ve senin uzak olmanı gerektiren şeyl...,['yapmam gereken ve senin uzak olmanı gerektir...,['ayrıca senden o burada değilken ailesine göz...
1997,çekip,33531,"['öylece çekip', 'buradan çekip', 'çekip gidem...","['çekip gidecek misin', 'sonra da çekip', 'öyl...","['öylece çekip gitmene izin', 'ayaklarınızı ma...","['senin iyi olacağını bilmeden çekip', 'iyi ol...","['senin iyi olacağını bilmeden çekip gidemem',...","['oradan çekip yerine başka bir şey koydu', 'ç...","['oradan çekip yerine başka bir şey koydu mu',...",['seni çekip çıkaracağım saruman bir yaradan z...,['ya parayı ve çekici alırsın ya da buradan çe...
1998,ekibi,33523,"['ku

In [25]:
df_twogram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx")
df_twogram = df_twogram.fillna("[]")
df_twogram = df_twogram.iloc[1:100,]
df_twogram

,twogram,frequency,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1,değil mi,585879,"['öyle değil mi', 'biliyorsun değil mi', 'güze...","['de öyle değil mi', 'bir şey değil mi', 'çok ...","['sence de öyle değil mi', 'sizce de öyle deği...","['bu iyi bir şey değil mi', 'ne demek istediği...","['o kadar da kötü değildi değil mi', 'o kadar ...",['bir kadının şarkı söylemesi oldukça şaşırtıc...,['evimde bir kadının şarkı söylemesi oldukça ş...,['ama bugün yeni bir maç var galiba değil mi m...
2,ben de,377765,"['ben de seni', 'ben de öyle', 've ben de', 'e...","['ben de seni seviyorum', 'ben de öyle düşünmü...","['ben de seni gördüğüme sevindim', 'evet ben d...","['ben de en az senin kadar', 'ben de sana aynı...","['ben de sizi karı koca ilan ediyorum', 'ben d...",['siobhan intihar etti ben de onun yerine geçt...,['siz formları alırsınız ben de onları hazırla...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,teşekkür ederim,370619,"['çok teşekkür ederim', 'için teşekkür ederim'...","['için çok teşekkür ederim', 'geldiğiniz için ...","['her şey için teşekkür ederim', 'zaman ayırdı...","['her şey için çok teşekkür ederim', 'beni kab...",['olmama izin verdiğin için tekrar teşekkür ed...,['gibi olmama izin verdiğin için tekrar teşekk...,['kendim gibi olmama izin verdiğin için tekrar...,['önce parfüm mağazasından alınmış video kaydı...
4,ne oldu,322758,"['sonra ne oldu', 'ne oldu sana', 'sana ne old...","['bil bakalım ne oldu', 'sana ne oldu böyle', ...","['az önce ne oldu öyle', 'sonra ne oldu biliyo...","['ne oldu bir sorun mu var', 'az önce ne oldu ...",['gelecek hakkındaki bilgileri reddetme konuşm...,['insanlık nereye gidiyoruz ve niçin soruların...,['ya insanlık nereye gidiyoruz ve niçin sorula...,['burnuna ne oldu gittes yoksa kapı yerine bac...
5,var mı,281902,"['şey var mı', 'kimse var mı', 'fikrin var mı'...","['bir şey var mı', 'bir fikrin var mı', 'bir y...","['başka bir şey var mı', 'yapabileceğim bir şe...","['söylemek istediğin bir şey var mı', 'daha iy...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['evet bu konu hakkında fikrini söylemek istey...,['aranızda bu adamların ateş etmeyi nerede öğr...
...,...,...,...,...,...,...,...,...,...,...
95,hadi gidelim,61646,"['hadi gidelim buradan', 'tamam hadi gidelim',...","['hadi gidelim o zaman', 'hadi gidelim hadi gi...","['hayalimdeki iş buydu hadi gidelim', 'hadi gi...","['hadi gidelim korkunç şeylerin olduğu yere', ...","['siz de ateşi üzerinize çekin hadi gidelim', ...",['arada siz de ateşi üzerinize çekin hadi gide...,['bu arada siz de ateşi üzerinize çekin hadi g...,['hadi gidelim bebeğim bebeğim oh biz tek bir ...
96,ama bu,61607,"['evet ama bu', 'ama bu sefer', 'ama bu çok', ...","['ama bu doğru değil', 'ama bu sadece bir', 'ö...","['ama bu uzun zaman önceydi', 'ama bu iyi bir ...","['bir sürü tüfek var ama bu', 'ama bu çok uzun...","['ama bu iyi bir şey değil mi', 'bir sürü tüfe...","['bir sürü tüfek var ama bu tüfek benim', 'söy...",['bunu söyleyip duruyorlar ama bu bizim için ö...,['sana yardım etmek isterim ama bu yüzden karı...
97,bir gün,61602,"['güzel bir gün', 'bir gün daha', 'belki bir g...","['bir gün değil mi', 'çok güzel bir gün', 'ne ...","['güzel bir gün değil mi', 'ne kadar güzel bir...","['ne güzel bir gün değil mi', 'neşeli ruhu bir...","['neşeli ruhu bir gün yelken açtı ta', 'ruhu b...",['neşeli ruhu bir gün yelken açtı ta denizden'...,['bir gün yalan açığa çıkacak ve karanlığın iç...,['bir gün yalan açığa çıkacak ve karanlığın iç...
98,iyi misin,61541,"['sen iyi misin', 'daha iyi misin', 'hey iyi m...","['hey sen iyi misin', 'şimdi daha iyi misin', ...","['hey hey sen iyi misin', 'gel buraya bebeğim ...","['ah iyi misin ahbap yapma bana', 'yerinde diy...","['de yerinde diye benden iyi misin yani', 'sab...",['bu sabah gerçekten tuhaf davranıyorsun sen i...,['bence ilişkimiz başladığından bu yana çok yo...,[]


In [27]:
df_threegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx")
df_threegram = df_threegram.fillna("[]")
df_threegram = df_threegram.iloc[1:100,]
df_threegram

,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1,bir şey var,110455,"['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'yapabileceğim bir şe...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['ev sahibi babamız için bende özel bir şey va...,['kaosun içinde düzen için mücadele eden tek b...
2,bu da ne,89463,"['bu da ne demek', 'bu da ne böyle', 'bu da ne...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu da ne demek oluyor şimdi', 'bu da ne deme...","['bu da ne biçim bir soru böyle', 'ama bu da n...","['alınmayın ama bu da ne demek oluyor şimdi', ...",[],['bu da ne olursa olsun oynamaya karar verdiği...
3,her şey yolunda,75968,"['her şey yolunda mı', 'her şey yolunda gidece...","['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda her şey yolunda', 'merak etm...",['helikopterler bizi koruyacak her şey yolunda...,"['her şey yolunda giderse çok yakında bir ev',...",['her şey yolunda giderse çok yakında bir ev a...,['dostlar acil bir durum yaşadık ama şimdi her...
4,başka bir şey,75193,"['başka bir şey var', 'başka bir şey yok', 'ba...","['başka bir şey var mı', 'başka bir şey daha v...","['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['kılıcını ve onun kılıcını düşün başka bir şe...,['sabırsızlıkla beklediğim zevkle yapacağım ba...
5,hayır hayır hayır,69443,"['hayır hayır hayır hayır', 'oh hayır hayır ha...","['hayır hayır hayır hayır hayır', 'oh hayır ha...","['hayır hayır hayır hayır hayır hayır', 'oh ha...","['hayır hayır hayır hayır hayır hayır hayır', ...",['hayır hayır hayır hayır hayır hayır hayır ha...,['hayır hayır hayır hayır hayır hayır hayır ha...,['hayır hayır hayır çok iyi evet bu oyun işe y...
...,...,...,...,...,...,...,...,...,...
95,çok büyük bir,16228,"['bu çok büyük bir', 'çok büyük bir hata', 'iç...","['çok büyük bir hata yapıyorsun', 'çok büyük b...","['bu benim için çok büyük bir', 'bir kişi için...","['bir kişi için çok büyük bir güç', 'önümüzdek...",['bu çok büyük bir saçmalık buraya bunun için'...,['çok büyük bir saçmalık buraya bunun için gel...,['bu çok büyük bir saçmalık buraya bunun için ...
96,çok daha iyi,16197,"['çok daha iyi bir', 'çok daha iyi hissediyoru...","['kendimi çok daha iyi hissediyorum', 'şimdi ç...","['şimdi kendimi çok daha iyi hissediyorum', 'd...",['oldukları yerden durumu bizden çok daha iyi'...,['oldukları yerden durumu bizden çok daha iyi ...,['ve umduğumdan da çok daha iyi bir jedi oldun...,['sanırım ben senden çok daha iyi bir özel aja...
97,peki ya sen,16151,"['peki ya sen ne', 'peki ya sen ve', 'peki ya ...","['peki ya sen ne yapıyordun', 'peki ya sen ne ...","['uğruna ölmeye hazırım peki ya sen', 'peki ya...","['ülkem uğruna ölmeye hazırım peki ya sen', 'p...",[],[],[]
98,iyi bir şey,16038,"['bu iyi bir şey', 'iyi bir şey değil', 'daha ...","['bu iyi bir şey değil', 'bu iyi bir şey mi', ...","['bu iyi bir şey değil mi', 'ama bu iyi bir şe...","['ama bu iyi bir şey değil mi', 'iyi bir şey m...","['iyi bir şey mi kötü bir şey mi', 'bu iyi bir...","['bu iyi bir şey mi kötü bir şey mi', 'çünkü e...",['sanırım ikimiz için de yalnız kalmak iyi bir...


In [12]:
df_fourgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx")
df_fourgram = df_fourgram.fillna("[]")
df_fourgram = df_fourgram.iloc[1000:1005,]
df_fourgram

,fourgram,frequency,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,hiç de iyi değil,851,"['bu hiç de iyi değil', 'için hiç de iyi değil...","['clem bu hiç de iyi değil', 'hiç de iyi değil...","['tadı hiç de iyi değil değil mi', 've onların...","['ve onların tadı hiç de iyi değil değil', 'on...",['ve onların tadı hiç de iyi değil değil mi'],[]
1001,çok ama çok üzgünüm,850,"['için çok ama çok üzgünüm', 'gerçekten çok am...","['olanlar için çok ama çok üzgünüm', 'isterim ...",['bilmeni isterim ki çok ama çok üzgünüm'],['önce bilmeni isterim ki çok ama çok üzgünüm'...,['ama önce bilmeni isterim ki çok ama çok üzgü...,['sebep olduğum bu korkunç sıkıntılar için çok...
1002,olur ne olmaz diye,849,"['ne olur ne olmaz diye', 'olur ne olmaz diye ...","['ama ne olur ne olmaz diye', 'ne olur ne olma...","['ne olur ne olmaz diye kuleye asker', 'buraya...",['şeyleri buraya saklamıştım ne olur ne olmaz ...,['bazı şeyleri buraya saklamıştım ne olur ne o...,[]
1003,benimle dalga geçiyor olmalısın,849,['benimle dalga geçiyor olmalısın dostum'],[],[],[],[],[]
1004,gereken bir şey daha,849,"['gereken bir şey daha var', 'bilmen gereken b...","['bilmen gereken bir şey daha var', 'yapmam ge...","['sana söylemem gereken bir şey daha var', 'am...","['ve sana vermem gereken bir şey daha var', 'g...",['yapmam gereken bir şey daha var sonra tamame...,['yapmam gereken bir şey daha var sonra tamame...


In [13]:
df_fivegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx")
df_fivegram = df_fivegram.fillna("[]")
df_fivegram = df_fivegram.iloc[1000:1005,]
df_fivegram

,fivegram,frequency,sixgram,sevengram,eightgram,ninegram,tengram
1000,ben de sana aynı şeyi,274,"['ben de sana aynı şeyi soracaktım', 'ben de s...",[],[],[],[]
1001,başka bir çıkış var mı,274,"['buradan başka bir çıkış var mı', 'bu başka b...","['bu başka bir çıkış var mı demek', 'başka bir...",['bu başka bir çıkış var mı demek oluyor'],[],[]
1002,jüri bir karara vardı mı,272,"['foreman jüri bir karara vardı mı', 'sözcü jü...","['bay foreman jüri bir karara vardı mı', 'sayı...",[],[],[]
1003,eden bir şey mi var,272,"['rahatsız eden bir şey mi var', 'eden bir şey...","['seni rahatsız eden bir şey mi var', 'sizi ra...","['seni rahatsız eden bir şey mi var oğlum', 's...",[],[]
1004,için yapabileceğim bir şey varsa,272,"['senin için yapabileceğim bir şey varsa', 'si...",['eğer senin için yapabileceğim bir şey varsa'...,['gitmeden önce senin için yapabileceğim bir ş...,['eğer gitmeden önce senin için yapabileceğim ...,['daha rahat etmeniz için yapabileceğim bir şe...


In [14]:
df_sixgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx")
df_sixgram = df_sixgram.fillna("[]")
df_sixgram = df_sixgram.iloc[1000:1005,]
df_sixgram

,sixgram,frequency,sevengram,eightgram,ninegram,tengram
1000,ama önce benim için bir şey,60,"['ama önce benim için bir şey yapman', 'ama ön...","['ama önce benim için bir şey yapman gerek', '...",[],[]
1001,söylemek istediğin bir şey mi vardı,60,['bana söylemek istediğin bir şey mi vardı'],['bana özellikle söylemek istediğin bir şey mi...,[],[]
1002,çöz kalbimi lütfen gel sal beni,60,['gel çöz kalbimi lütfen gel sal beni'],[],[],[]
1003,bana şaplak atmana izin vermek hoşuma,60,['bana şaplak atmana izin vermek hoşuma gider'],[],[],[]
1004,yürüyüşüne devam et ve hep kendine,60,['kendince yürüyüşüne devam et ve hep kendine'...,['kendince yürüyüşüne devam et ve hep kendine ...,['sadece kendince yürüyüşüne devam et ve hep k...,[]


In [15]:
df_sevengram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx")
df_sevengram = df_sevengram.fillna("[]")
df_sevengram = df_sevengram.iloc[1000:1005,]
df_sevengram

,sevengram,frequency,eightgram,ninegram,tengram
1000,en önemli şey daima buradaki insanlar olacak,35,['hayattaki en önemli şey daima buradaki insan...,['ama hayattaki en önemli şey daima buradaki i...,[]
1001,ismi gelen iki pisliği geri götürmeye geldim,35,['masama ismi gelen iki pisliği geri götürmeye...,[],[]
1002,iki adam hakkında sana bir şey diyeyim,35,['bu iki adam hakkında sana bir şey diyeyim'],[],[]
1003,dostum işinizin ehli olduğunuzu biliyor ve saygı,35,['bak dostum işinizin ehli olduğunuzu biliyor ...,['bak dostum işinizin ehli olduğunuzu biliyor ...,[]
1004,şüphe yok ki gemiyi ve bizim hayatlarımızı,35,['hiç şüphe yok ki gemiyi ve bizim hayatlarımı...,['hiç şüphe yok ki gemiyi ve bizim hayatlarımı...,[]


In [16]:
df_eightgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx")
df_eightgram = df_eightgram.fillna("[]")
df_eightgram = df_eightgram.iloc[1000:1005,]
df_eightgram

,eightgram,frequency,ninegram,tengram
1000,düşünmüştüm ki sabah uyandığımda kendimi daha ...,29,['bugün düşünmüştüm ki sabah uyandığımda kendi...,[]
1001,çıkış noktası hakkında hiçbir şey bilmiyorsun ...,29,['müziğin çıkış noktası hakkında hiçbir şey bi...,['hiphop müziğin çıkış noktası hakkında hiçbir...
1002,olduğun yerde dur ve ellerini görünür şekilde tut,29,['zaman olduğun yerde dur ve ellerini görünür ...,['o zaman olduğun yerde dur ve ellerini görünü...
1003,ömür boyu içeride kalmamızı sağlayacak kadar t...,29,['ömür boyu içeride kalmamızı sağlayacak kadar...,[]
1004,adamla aranızda kan bağı vardır zira yaptığın ...,29,['umarım adamla aranızda kan bağı vardır zira ...,[]


In [17]:
df_ninegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx")
df_ninegram = df_ninegram.fillna("[]")
df_ninegram = df_ninegram.iloc[1000:1005,]
df_ninegram

,ninegram,frequency,tengram
1000,gün devam ederken david bunun en mutlu günü ol...,21,['gün devam ederken david bunun en mutlu günü ...
1001,tek istediğim ülkeme dönmek ve buna param yüz ...,21,['tek istediğim ülkeme dönmek ve buna param yü...
1002,gidip masaya oturmanı ve oraya bir isim yazman...,21,['mutfağa gidip masaya oturmanı ve oraya bir i...
1003,hanımlar kapıyı kapatıp bizi bir süreliğine ya...,21,['müsaadenizle hanımlar kapıyı kapatıp bizi bi...
1004,pilotun her an üzerinde olması gereken iki şey...,21,['bir pilotun her an üzerinde olması gereken i...


#### Sending Data To Firestore

##### Photo Data

In [ ]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Text Data

##### Word

In [22]:
df_word["twogram"] = df_word["twogram"].apply(eval)
df_word["threegram"] = df_word["threegram"].apply(eval)
df_word["fourgram"] = df_word["fourgram"].apply(eval)
df_word["fivegram"] = df_word["fivegram"].apply(eval)
df_word["sixgram"] = df_word["sixgram"].apply(eval)
df_word["sevengram"] = df_word["sevengram"].apply(eval)
df_word["eightgram"] = df_word["eightgram"].apply(eval)
df_word["ninegram"] = df_word["ninegram"].apply(eval)
df_word["tengram"] = df_word["tengram"].apply(eval)

In [ ]:
word = df_word["word"].to_list()
frequency_list = df_word["frequency"].to_list()
twogram_list = df_word["twogram"].to_list()
threegram_list = df_word["threegram"].to_list()
fourgram_list = df_word["fourgram"].to_list()
fivegram_list = df_word["fivegram"].to_list()
sixgram_list = df_word["sixgram"].to_list()
sevengram_list = df_word["sevengram"].to_list()
eightgram_list = df_word["eightgram"].to_list()
ninegram_list = df_word["ninegram"].to_list()
tengram_list = df_word["tengram"].to_list()

In [ ]:
firestore_word_data = zip(word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
#word_data_list = []
#for word,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
#    data = {f"{word}":{"ninegram":ninegram_list,"tengram":tengram_list}}
#    
#    word_data_list.append({f"{word}":data})

In [ ]:
word_dict = {}
for word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
    dict_var = {f"{word}":{"photo":"","frequency":frequency_list,"twogram":twogram_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    word_dict.update(dict_var)     

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").set(word_dict)

In [ ]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").add(word_dict)

##### Update

In [ ]:
update_word = "hoşuma"

In [ ]:
dict_update = {f"{update_word}.photo":"www.store4.com",f"{update_word}.other":["gitti",{"zıt":"gelmek"}]}

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").update(dict_update)

##### Twogram

In [ ]:
df_twogram

In [26]:
df_twogram["threegram"] = df_twogram["threegram"].apply(eval)
df_twogram["fourgram"] = df_twogram["fourgram"].apply(eval)
df_twogram["fivegram"] = df_twogram["fivegram"].apply(eval)
df_twogram["sixgram"] = df_twogram["sixgram"].apply(eval)
df_twogram["sevengram"] = df_twogram["sevengram"].apply(eval)
df_twogram["eightgram"] = df_twogram["eightgram"].apply(eval)
df_twogram["ninegram"] = df_twogram["ninegram"].apply(eval)
df_twogram["tengram"] = df_twogram["tengram"].apply(eval)

SyntaxError: unexpected EOF while parsing (<string>, line 1)

In [20]:
#twogram = df_twogram["twogram"].values.tolist()

In [ ]:
twogram = df_twogram["twogram"].to_list()
frequency_list = df_twogram["frequency"].to_list()
threegram_list = df_twogram["threegram"].to_list()
fourgram_list = df_twogram["fourgram"].to_list()
fivegram_list = df_twogram["fivegram"].to_list()
sixgram_list = df_twogram["sixgram"].to_list()
sevengram_list = df_twogram["sevengram"].to_list()
eightgram_list = df_twogram["eightgram"].to_list()
ninegram_list = df_twogram["ninegram"].to_list()
tengram_list = df_twogram["tengram"].to_list()

In [ ]:
firestore_twogram_data = zip(twogram,frequency_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
twogram_dict = {}
for twogram,frequency_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_twogram_data:
    dict_var = {f"{twogram}":{"photo":"","frequency":frequency_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    twogram_dict.update(dict_var)    

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("twogram_data").document(f"details").set(twogram_dict)

##### Threegram

In [28]:
df_threegram

,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1,bir şey var,110455,"['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'yapabileceğim bir şe...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['ev sahibi babamız için bende özel bir şey va...,['kaosun içinde düzen için mücadele eden tek b...
2,bu da ne,89463,"['bu da ne demek', 'bu da ne böyle', 'bu da ne...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu da ne demek oluyor şimdi', 'bu da ne deme...","['bu da ne biçim bir soru böyle', 'ama bu da n...","['alınmayın ama bu da ne demek oluyor şimdi', ...",[],['bu da ne olursa olsun oynamaya karar verdiği...
3,her şey yolunda,75968,"['her şey yolunda mı', 'her şey yolunda gidece...","['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda her şey yolunda', 'merak etm...",['helikopterler bizi koruyacak her şey yolunda...,"['her şey yolunda giderse çok yakında bir ev',...",['her şey yolunda giderse çok yakında bir ev a...,['dostlar acil bir durum yaşadık ama şimdi her...
4,başka bir şey,75193,"['başka bir şey var', 'başka bir şey yok', 'ba...","['başka bir şey var mı', 'başka bir şey daha v...","['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['kılıcını ve onun kılıcını düşün başka bir şe...,['sabırsızlıkla beklediğim zevkle yapacağım ba...
5,hayır hayır hayır,69443,"['hayır hayır hayır hayır', 'oh hayır hayır ha...","['hayır hayır hayır hayır hayır', 'oh hayır ha...","['hayır hayır hayır hayır hayır hayır', 'oh ha...","['hayır hayır hayır hayır hayır hayır hayır', ...",['hayır hayır hayır hayır hayır hayır hayır ha...,['hayır hayır hayır hayır hayır hayır hayır ha...,['hayır hayır hayır çok iyi evet bu oyun işe y...
...,...,...,...,...,...,...,...,...,...
95,çok büyük bir,16228,"['bu çok büyük bir', 'çok büyük bir hata', 'iç...","['çok büyük bir hata yapıyorsun', 'çok büyük b...","['bu benim için çok büyük bir', 'bir kişi için...","['bir kişi için çok büyük bir güç', 'önümüzdek...",['bu çok büyük bir saçmalık buraya bunun için'...,['çok büyük bir saçmalık buraya bunun için gel...,['bu çok büyük bir saçmalık buraya bunun için ...
96,çok daha iyi,16197,"['çok daha iyi bir', 'çok daha iyi hissediyoru...","['kendimi çok daha iyi hissediyorum', 'şimdi ç...","['şimdi kendimi çok daha iyi hissediyorum', 'd...",['oldukları yerden durumu bizden çok daha iyi'...,['oldukları yerden durumu bizden çok daha iyi ...,['ve umduğumdan da çok daha iyi bir jedi oldun...,['sanırım ben senden çok daha iyi bir özel aja...
97,peki ya sen,16151,"['peki ya sen ne', 'peki ya sen ve', 'peki ya ...","['peki ya sen ne yapıyordun', 'peki ya sen ne ...","['uğruna ölmeye hazırım peki ya sen', 'peki ya...","['ülkem uğruna ölmeye hazırım peki ya sen', 'p...",[],[],[]
98,iyi bir şey,16038,"['bu iyi bir şey', 'iyi bir şey değil', 'daha ...","['bu iyi bir şey değil', 'bu iyi bir şey mi', ...","['bu iyi bir şey değil mi', 'ama bu iyi bir şe...","['ama bu iyi bir şey değil mi', 'iyi bir şey m...","['iyi bir şey mi kötü bir şey mi', 'bu iyi bir...","['bu iyi bir şey mi kötü bir şey mi', 'çünkü e...",['sanırım ikimiz için de yalnız kalmak iyi bir...


In [29]:
df_threegram["fourgram"] = df_threegram["fourgram"].apply(eval)
df_threegram["fivegram"] = df_threegram["fivegram"].apply(eval)
df_threegram["sixgram"] = df_threegram["sixgram"].apply(eval)
df_threegram["sevengram"] = df_threegram["sevengram"].apply(eval)
df_threegram["eightgram"] = df_threegram["eightgram"].apply(eval)
df_threegram["ninegram"] = df_threegram["ninegram"].apply(eval)
df_threegram["tengram"] = df_threegram["tengram"].apply(eval)

SyntaxError: EOL while scanning string literal (<string>, line 1)

In [ ]:
threegram = df_threegram["threegram"].to_list()
frequency_list = df_threegram["frequency"].to_list()
fourgram_list = df_threegram["fourgram"].to_list()
fivegram_list = df_threegram["fivegram"].to_list()
sixgram_list = df_threegram["sixgram"].to_list()
sevengram_list = df_threegram["sevengram"].to_list()
eightgram_list = df_threegram["eightgram"].to_list()
ninegram_list = df_threegram["ninegram"].to_list()
tengram_list = df_threegram["tengram"].to_list()

In [ ]:
firestore_threegram_data = zip(threegram,frequency_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
threegram_dict = {}
for threegram,frequency_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_threegram_data:
    dict_var = {f"{threegram}":{"photo":"","frequency":frequency_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    threegram_dict.update(dict_var)    

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("threegram_data").document(f"details").set(threegram_dict)

##### Fourgram

In [ ]:
df_fourgram

In [ ]:
df_fourgram["fivegram"] = df_fourgram["fivegram"].apply(eval)
df_fourgram["sixgram"] = df_fourgram["sixgram"].apply(eval)
df_fourgram["sevengram"] = df_fourgram["sevengram"].apply(eval)
df_fourgram["eightgram"] = df_fourgram["eightgram"].apply(eval)
df_fourgram["ninegram"] = df_fourgram["ninegram"].apply(eval)
df_fourgram["tengram"] = df_fourgram["tengram"].apply(eval)

In [ ]:
fourgram = df_fourgram["fourgram"].to_list()
frequency_list = df_fourgram["frequency"].to_list()
fivegram_list = df_fourgram["fivegram"].to_list()
sixgram_list = df_fourgram["sixgram"].to_list()
sevengram_list = df_fourgram["sevengram"].to_list()
eightgram_list = df_fourgram["eightgram"].to_list()
ninegram_list = df_fourgram["ninegram"].to_list()
tengram_list = df_fourgram["tengram"].to_list()

In [ ]:
firestore_fourgram_data = zip(fourgram,frequency_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
fourgram_dict = {}
for fourgram,frequency_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fourgram_data:
    dict_var = {f"{fourgram}":{"photo":"","frequency":frequency_list,"fivegram":fivegram_list,"sixgram":sixgram_list,
    "sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    fourgram_dict.update(dict_var)    

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fourgram_data").document(f"details").set(fourgram_dict)

##### Fivegram

In [ ]:
df_fivegram

In [ ]:
df_fivegram["sixgram"] = df_fivegram["sixgram"].apply(eval)
df_fivegram["sevengram"] = df_fivegram["sevengram"].apply(eval)
df_fivegram["eightgram"] = df_fivegram["eightgram"].apply(eval)
df_fivegram["ninegram"] = df_fivegram["ninegram"].apply(eval)
df_fivegram["tengram"] = df_fivegram["tengram"].apply(eval)

In [ ]:
fivegram = df_fivegram["fivegram"].to_list()
frequency_list = df_fivegram["frequency"].to_list()
sixgram_list = df_fivegram["sixgram"].to_list()
sevengram_list = df_fivegram["sevengram"].to_list()
eightgram_list = df_fivegram["eightgram"].to_list()
ninegram_list = df_fivegram["ninegram"].to_list()
tengram_list = df_fivegram["tengram"].to_list()

In [ ]:
firestore_fivegram_data = zip(fivegram,frequency_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
fivegram_dict = {}
for fivegram,frequency_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fivegram_data:
    dict_var = {f"{fivegram}":{"photo":"","frequency":frequency_list,"sixgram":sixgram_list,
    "sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    fivegram_dict.update(dict_var)   

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fivegram_data").document(f"details").set(fivegram_dict)

##### Sixgram

In [ ]:
df_sixgram

In [ ]:
df_sixgram["sevengram"] = df_sixgram["sevengram"].apply(eval)
df_sixgram["eightgram"] = df_sixgram["eightgram"].apply(eval)
df_sixgram["ninegram"] = df_sixgram["ninegram"].apply(eval)
df_sixgram["tengram"] = df_sixgram["tengram"].apply(eval)

In [ ]:
sixgram = df_sixgram["sixgram"].to_list()
frequency_list = df_sixgram["frequency"].to_list()
sevengram_list = df_sixgram["sevengram"].to_list()
eightgram_list = df_sixgram["eightgram"].to_list()
ninegram_list = df_sixgram["ninegram"].to_list()
tengram_list = df_sixgram["tengram"].to_list()

In [ ]:
firestore_sixgram_data = zip(sixgram,frequency_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
sixgram_dict = {}
for sixgram,frequency_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_sixgram_data:
    dict_var = {f"{sixgram}":{"photo":"","frequency":frequency_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    sixgram_dict.update(dict_var)   

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sixgram_data").document(f"details").set(sixgram_dict)

##### Sevengram

In [ ]:
df_sevengram

In [ ]:
df_sevengram["eightgram"] = df_sevengram["eightgram"].apply(eval)
df_sevengram["ninegram"] = df_sevengram["ninegram"].apply(eval)
df_sevengram["tengram"] = df_sevengram["tengram"].apply(eval)

In [ ]:
sevengram = df_sevengram["sevengram"].to_list()
frequency_list = df_sevengram["frequency"].to_list()
eightgram_list = df_sevengram["eightgram"].to_list()
ninegram_list = df_sevengram["ninegram"].to_list()
tengram_list = df_sevengram["tengram"].to_list()

In [ ]:
firestore_sevengram_data = zip(sevengram,frequency_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
sevengram_dict = {}
for sevengram,frequency_list,eightgram_list,ninegram_list,tengram_list in firestore_sevengram_data:
    dict_var = {f"{sevengram}":{"photo":"","frequency":frequency_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    sevengram_dict.update(dict_var)  

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sevengram_data").document(f"details").set(sevengram_dict)

##### Eightgram

In [ ]:
df_eightgram

In [ ]:
df_eightgram["ninegram"] = df_eightgram["ninegram"].apply(eval)
df_eightgram["tengram"] = df_eightgram["tengram"].apply(eval)

In [ ]:
eightgram = df_eightgram["eightgram"].to_list()
frequency_list = df_eightgram["frequency"].to_list()
ninegram_list = df_eightgram["ninegram"].to_list()
tengram_list = df_eightgram["tengram"].to_list()

In [ ]:
firestore_eightgram_data = zip(eightgram,frequency_list,ninegram_list,tengram_list)

In [ ]:
eightgram_dict = {}
for eightgram,frequency_list,ninegram_list,tengram_list in firestore_eightgram_data:
    dict_var = {f"{eightgram}":{"photo":"","frequency":frequency_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    eightgram_dict.update(dict_var)  

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("eightgram_data").document(f"details").set(eightgram_dict)

##### Ninegram

In [ ]:
df_ninegram

In [ ]:
df_ninegram["tengram"] = df_ninegram["tengram"].apply(eval)

In [ ]:
ninegram = df_ninegram["ninegram"].to_list()
frequency_list = df_ninegram["frequency"].to_list()
tengram_list = df_ninegram["tengram"].to_list()

In [ ]:
firestore_ninegram_data = zip(ninegram,frequency_list,tengram_list)

In [ ]:
ninegram_dict = {}
for ninegram,frequency_list,tengram_list in firestore_ninegram_data:
    dict_var = {f"{ninegram}":{"photo":"","frequency":frequency_list,"tengram":tengram_list,"other":[]}}
    ninegram_dict.update(dict_var)  

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("ninegram_data").document(f"details").set(ninegram_dict)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass